In [1]:
import numpy as np
import gym

In [86]:
env = gym.make('FrozenLake-v0', is_slippery=False)

In [48]:
len(env.P), len(env.P[0]), len(env.P[0][0])

(16, 4, 3)

In [ ]:
def __init__(self, env):
    #This code would be modified to make more generalized
    self.actions_n = env.nA
    self.states_n = env.nS
    self.R = np.zeros([self.states_n, self.actions_n, self.states_n])
    self.T = np.zeros([self.states_n, self.actions_n, self.states_n])
    for s in range(self.states_n):
        for a in range(self.actions_n):
            t = env.P[s][a]
            for prob_trans, next_state, reward, done in t:
                self.T[s,a,next_state] += prob_trans
                if done and reward == 0.0:
                    reward = -.75
                self.R[s,a,next_state] = reward

            #Normalize T
            self.T[s,a,:]/=np.sum(self.T[s,a,:])
                
                
nA, nS = env.nA, env.nS
P = np.zeros([nA, nS, nS])
R = np.zeros([nS, nA])
for s in range(nS):
    for a in range(nA):
        transitions = env.P[s][a]
        for p_trans, next_s, reward, _ in transitions:
            P[a,s,next_s] += p_trans
            R[s,a] = reward
        P[a,s,:] /= np.sum(P[a,s,:])

In [47]:
env.P[15]

{0: [(1.0, 15, 0, True)],
 1: [(1.0, 15, 0, True)],
 2: [(1.0, 15, 0, True)],
 3: [(1.0, 15, 0, True)]}

In [87]:
nA, nS = env.nA, env.nS
P = np.zeros([nA, nS, nS])
R = np.zeros([nS, nA])

for s in range(nS):
    for a in range(nA):
        transitions = env.P[s][a]
        for prob, next_s, reward, done in transitions:
            P[a,s,next_s] += prob
            R[s, a] = reward
        P[a,s,:] /= np.sum(P[a,s,:])
        

In [72]:
import hiive.mdptoolbox.mdp as mdp

In [88]:
vi = mdp.ValueIteration(P, R, 0.9)

In [89]:
vi.run()

[{'State': None,
  'Action': None,
  'Reward': 1.0,
  'Error': 1.0,
  'Time': 0.0012514591217041016,
  'Max V': 1.0,
  'Mean V': 0.0625,
  'Iteration': 1},
 {'State': None,
  'Action': None,
  'Reward': 1.0,
  'Error': 0.9,
  'Time': 0.0014176368713378906,
  'Max V': 1.0,
  'Mean V': 0.175,
  'Iteration': 2},
 {'State': None,
  'Action': None,
  'Reward': 1.0,
  'Error': 0.81,
  'Time': 0.0017397403717041016,
  'Max V': 1.0,
  'Mean V': 0.27625,
  'Iteration': 3},
 {'State': None,
  'Action': None,
  'Reward': 1.0,
  'Error': 0.7290000000000001,
  'Time': 0.0018434524536132812,
  'Max V': 1.0,
  'Mean V': 0.367375,
  'Iteration': 4},
 {'State': None,
  'Action': None,
  'Reward': 1.0,
  'Error': 0.6561000000000001,
  'Time': 0.0019197463989257812,
  'Max V': 1.0,
  'Mean V': 0.49039375,
  'Iteration': 5},
 {'State': None,
  'Action': None,
  'Reward': 1.0,
  'Error': 0.5904900000000002,
  'Time': 0.002004861831665039,
  'Max V': 1.0,
  'Mean V': 0.5272993750000001,
  'Iteration': 6},
 

In [90]:
policy = np.asarray(vi.policy).reshape((4,4))
print(policy)
print(np.asarray(convert_policy_string(policy)))

[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


[['v' '>' 'v' '<']
 ['v' '<' 'v' '<']
 ['>' 'v' 'v' '<']
 ['<' '>' '>' '<']]


In [97]:
pi = mdp.PolicyIteration(P, R, 0.9)
pi.run()
pi.policy

(1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0)

In [98]:
policy = np.asarray(pi.policy).reshape((4,4))
print(policy)
print(np.asarray(convert_policy_string(policy)))

[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]
[['v' '>' 'v' '<']
 ['v' '<' 'v' '<']
 ['>' 'v' 'v' '<']
 ['<' '>' '>' '<']]


In [105]:
ql = mdp.QLearning(P, R, 0.96,epsilon=0.8, n_iter=100000)
ql.run()
ql.policy

(1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0)

In [106]:
policy = np.asarray(ql.policy).reshape((4,4))
print(policy)
print(np.asarray(convert_policy_string(policy)))

[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]
[['v' '>' 'v' '<']
 ['v' '<' 'v' '<']
 ['>' 'v' 'v' '<']
 ['<' '>' '>' '<']]


In [ ]:
def __init__(self, transitions, reward, gamma,
             alpha=0.1, alpha_decay=0.99, alpha_min=0.001,
             epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.99,
             n_iter=10000, skip_check=False, iter_callback=None,
             run_stat_frequency=None):

<!-- # def __init__(self, transitions, reward, gamma,
#              alpha=0.1, alpha_decay=0.99, alpha_min=0.001,
#              epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.99,
#              n_iter=10000, skip_check=False, iter_callback=None,
#              run_stat_frequency=None): -->

In [ ]:
ql = mdptoolbox.mdp.QLearning(P, R, 0.96)
    >>> ql.run()
    >>> ql.Q
        >>> expected = (11.198908998901134, 11.741057920409865, 12.259732864170232)
    >>> all(expected[k] - ql.V[k] < 1e-12 for k in range(len(expected)))
    True
    >>> ql.policy

In [49]:
env.P[0]

{0: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False)],
 1: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 1, 0.0, False)],
 2: [(0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)]}

In [61]:
for a,b,c,d in env.P[5][0]:
    print(a,b,c,d)

1.0 5 0 True


In [62]:
for k in env.P:
    for a in env.P[k]:
        for a,b,c,d in env.P[k][a]:
            print(a,b,c,d)

0.3333333333333333 0 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 4 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 4 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 4 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 5 0.0 True
0.3333333333333333 0 0.0 False
0.3333333333333333 5 0.0 True
0.3333333333333333 2 0.0 False
0.3333333333333333 5 0.0 True
0.3333333333333333 2 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 2 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 0 0.0 False
0.3333333333333333 2 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 6 0.0 False
0.3333333333333333 1 0.0 False
0.3333333333333333 6 0.0 False
0.3333333333333333 3 0.0 False
0.3333333333333333 6 0.0 False
0.3333333333333333 3 0.0 False
0.333333333

In [77]:
def convert_policy_string(policy, action_str=['<','v','>','^']):
    # action_str: ['<','v','>','^'] 0=left, 1=down, 2=right, 3=up
    
    p = [[' ' for j in range(len(policy[0]))] for i in range(len(policy))]
    
    for i in range(policy.shape[0]):
        for j in range(policy.shape[1]):
            p[i][j] = action_str[policy[i,j]]
            
    return p

In [78]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            v = V[s]
            action_values = np.zeros(env.nA)
            for a in range(env.nA):
                next_state_value = 0
                for prob, next_state, reward, done in env.P[s][a]:
                    next_state_value += prob * (reward + discount_factor * V[next_state])
                action_values[a] = next_state_value
            best_action_value = np.max(action_values)
            V[s] = best_action_value

            delta = max(delta, np.abs(v - V[s]))
        if (delta < theta):
            print("converged")
            break

    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        action_values = np.zeros(env.nA)
        for a in range(env.nA):
            next_state_value = 0
            for prob, next_state, reward, done in env.P[s][a]:
                next_state_value += prob * (reward + discount_factor * V[next_state])
            action_values[a] = next_state_value
        best_action = np.argmax(action_values)
        policy[s, best_action] = 1.0

    return policy, V



In [8]:
discount_factor = 0.9



env = gym.make('FrozenLake-v0', is_slippery=False)
env.render()
env.reset()
p, v = value_iteration(env, theta=0.0001, discount_factor=discount_factor)
print(v)
print("Reshaped Grid Policy (0=left, 1=down, 2=right, 3=up):")
print(np.reshape(np.argmax(p, axis=1), (4,4)))



SFFF
FHFH
FFFH
HFFG
converged
[0.59049 0.6561  0.729   0.6561  0.6561  0.      0.81    0.      0.729
 0.81    0.9     0.      0.      0.9     1.      0.     ]
Reshaped Grid Policy (0=left, 1=down, 2=right, 3=up):
[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


In [12]:
policy = np.reshape(np.argmax(p, axis=1), (4,4))

In [61]:
a = ['<','v','>','^'] # 0=left, 1=down, 2=right, 3=up

In [62]:

for i in range(policy.shape[0]):
    for j in range(policy.shape[1]):
        b[i][j] = a[policy[i,j]]
print(np.asarray(b))

[['v' '>' 'v' '<']
 ['v' '<' 'v' '<']
 ['>' 'v' 'v' '<']
 ['<' '>' '>' '<']]


In [76]:
policy

array([[1, 2, 1, 0],
       [1, 0, 1, 0],
       [2, 1, 1, 0],
       [0, 2, 2, 0]])

In [78]:
print(np.asarray(convert_policy_string(policy)))

[['v' '>' 'v' '<']
 ['v' '<' 'v' '<']
 ['>' 'v' 'v' '<']
 ['<' '>' '>' '<']]


In [74]:
[[' ' for j in range(len(policy[0]))] for i in range(len(policy))]

[[' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ']]

In [71]:
len(a), len(a[0])

(3, 4)